# A/B tests с Python




<a id="Libraries"></a>
##  Загрузка библиотек 

In [87]:
#pip install plotly

In [88]:
#pip install statsmodels

In [89]:
#pip install openpyxl

In [90]:
from math import asin
from typing import Union

import pandas as pd
import numpy as np
import plotly.express as px

from scipy import stats
from statsmodels.stats.meta_analysis import effectsize_smd
from statsmodels.stats import proportion
from statsmodels.stats.power import tt_ind_solve_power
from statsmodels.stats.power import zt_ind_solve_power

# Подготовка данных

In [91]:
#читаем excel
df = pd.read_excel('gb_sem_8_hm.xlsx')

C:\Users\Geralt\tmp\.env\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning:

Unknown extension is not supported and will be removed



In [92]:
#выодим первые 10 строк
df.head(10)

,USER_ID,VARIANT_NAME,REVENUE
0,737,variant,0.0
1,2423,control,0.0
2,9411,control,0.0
3,7311,control,0.0
4,6174,variant,0.0
5,2380,variant,0.0
6,2849,control,0.0
7,9168,control,0.0
8,6205,variant,0.0
9,7548,control,0.0


In [93]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   USER_ID       10000 non-null  int64  
 1   VARIANT_NAME  10000 non-null  object 
 2   REVENUE       10000 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 234.5+ KB


In [94]:
# смотрим общее количество строк
df.shape

(10000, 3)

In [95]:
# посмотрим количество уникальны id пользователей
df.USER_ID.nunique()

6324

In [96]:
# т.к. количество уникальных id пользователей меньше, есть не уникальные id, сгруппируем данные по сумме выручки
df = df.groupby(['USER_ID', 'VARIANT_NAME'], as_index=False).agg({'REVENUE': 'sum'})

In [97]:
# видим, что теперь количество строк 7865, что меньше количества уникальных id. Что означает, чо некоторые пользователи попали и в тестовую и в контрольную группу
data.shape

(7865, 3)

In [98]:
#определим количество пользователей, попавших в обе группы - 1541
df.groupby('USER_ID', as_index=False).agg({'VARIANT_NAME': 'count'})['VARIANT_NAME'].value_counts()

1    4783
2    1541
Name: VARIANT_NAME, dtype: int64

In [99]:
# оставим только набблюдения для пользователей не попавших в обе группы
unique_ids = \
(df
 .groupby('USER_ID', as_index=False)
 .agg({'VARIANT_NAME': 'count'})
 #.['VARIANT_NAME'].value_counts()
 .query('VARIANT_NAME == 1')
 .USER_ID
 .values
 )

In [100]:
# оставим только набблюдения для пользователей не попавших в обе группы
df_new = df[df.USER_ID.isin(unique_ids)].copy(deep=True)

In [101]:
# разделим данные на две группы: тестовую и контрольную
control = df_new.query('VARIANT_NAME == "control"')

test = df_new.query('VARIANT_NAME == "variant"')

выведем описание данных в контрольной и тестовой группе и график

In [102]:
control.describe()

,USER_ID,REVENUE
count,2390.000000,2390.000000
mean,5020.881590,0.196887
std,2904.850992,4.172201
min,2.000000,0.000000
25%,2517.250000,0.000000
50%,5012.500000,0.000000
75%,7616.000000,0.000000
max,9998.000000,196.010000


In [103]:
test.describe()

,USER_ID,REVENUE
count,2393.000000,2393.000000
mean,4967.943168,0.074935
std,2892.745368,0.858207
min,4.000000,0.000000
25%,2435.000000,0.000000
50%,4955.000000,0.000000
75%,7379.000000,0.000000
max,9995.000000,23.040000


In [104]:
control.describe()
test.describe()

fig = px.histogram(df_new[df_new.REVENUE.between(*df_new[df_new.REVENUE != 0].quantile(q=[0, .99]).REVENUE.values)],
                   x='REVENUE',
                   color = 'VARIANT_NAME',
                   marginal = 'box',
                   nbins = 10)
fig.show()

C:\Users\Geralt\AppData\Local\Temp\ipykernel_15948\2132400556.py:4: FutureWarning:

The default value of numeric_only in DataFrame.quantile is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.



Видно, что в обоих группах большая дисперсия. При этом средняя выручка в контрольной группе больше.

# Проверка гипотезы

Мы должны проверить, являестя ли различие в редней ыручке статистически значимым. Для этого:
- проверить данные на нормальность распределения,
- выбрать статистический критерий для проверки гипотезы,
- посчитать статистику

In [105]:
#функция для проверки на нормальность распределения
def is_normal_distr(x):
    
    if x.size < 5_000:
        _, pvalue = stats.shapiro(x)
    else:
        _, pvalue = stats.kstest(x, 'norm')
        
    return pvalue

In [106]:
#функция для расчета статистики (для количественной т.е. непрерывной метрики, коей являеется выручка)
def calc_continuous_effect(control: pd.DataFrame,
                           treatment: pd.DataFrame,
                           column: str,
                           stat_test: Union['t', 'mw'] = 't') -> pd.DataFrame:
    
    control_mean = control.loc[:, column].mean()
    treatment_mean = treatment.loc[:, column].mean()
    
    control_std = control.loc[:, column].std(ddof=1)
    treatment_std = treatment.loc[:, column].std(ddof=1)
    
    nobs1 = control.shape[0]
    nobs2 = treatment.shape[0]
    
    # effect_size = (treatment_mean - control_mean) / ((control_std ** 2 + treatment_std ** 2) / 2) ** .5
    effect_size, _ = effectsize_smd(mean1=treatment_mean, sd1=treatment_std, nobs1=nobs2,
                                    mean2=control_mean, sd2=control_std, nobs2=nobs1)
    
    if stat_test == 't':
        _, pvalue = stats.ttest_ind(a=control.loc[:, column],
                                    b=treatment.loc[:, column],
                                    equal_var=False, # perform Welch's t-test
                                    alternative='two-sided')
   
    elif stat_test == 'mw':
        _, pvalue = stats.mannwhitneyu(x=control.loc[:, column],
                                       y=treatment.loc[:, column],
                                       alternative='two-sided')
    else:
        raise NotImplementedError()

    power = tt_ind_solve_power(effect_size=effect_size,
                               nobs1=control.shape[0],
                               alpha=pvalue,
                               power=None,
                               ratio=nobs2/nobs1)
    
    pw_settings = {'alpha': .05, 'power': .8}
    pw_nobs = tt_ind_solve_power(effect_size=effect_size,
                                 nobs1=None,
                                 alpha=pw_settings['alpha'],
                                 power=pw_settings['power'],
                                 ratio=1)
    
    difference = treatment_mean - control_mean
    
    result = pd.DataFrame({'effect_size': effect_size,
                           'alpha': pvalue, 
                           'beta': (1-power),
                           'power': power,
                           'difference': difference,
                           'nobs': nobs1 + nobs2},
                          index=[column]) 
    
    perfect_way = pd.DataFrame({'effect_size': effect_size,
                                'alpha': pw_settings['alpha'],
                                'beta': 1 - pw_settings['power'],
                                'power': pw_settings['power'],
                                'difference': difference,
                                'nobs': round(pw_nobs * 2, 0)},
                               index=['perfect_way'])
    
    return pd.concat((result, perfect_way))

# Проверка на нормальность распределения и применение статистических критериев

In [107]:
#Вызываем функцию проверки распределения на нормальность
column = 'REVENUE'
control_is_normal = is_normal_distr(control.loc[:, column])
test_is_normal = is_normal_distr(test.loc[:, column])
control_is_normal, test_is_normal

(0.0, 0.0)

Т.к. в обеих группах полученные значения p-value меньше уровня значимости, то принимаем альтернативную гипотезу, что распределение НЕ является нормальным.

In [108]:
#Вызывваем фунцию расчета статистики, т.к. распределение не нормальное, то выбираем критерий Манна-Уитни
calc_continuous_effect(control, test, column='REVENUE', stat_test='mw')

,effect_size,alpha,beta,power,difference,nobs
REVENUE,-0.040495,0.210488,0.437197,0.562803,-0.121952,4783.0
perfect_way,-0.040495,0.050000,0.200000,0.800000,-0.121952,19148.0


Выводы на основе статистики:
- альфа больше 0.5, что показывает, что мы должны принять нулевую гипотезу о том, что занчимой разницы между группы
- бета (вероятность ошибки второго рода, вероятность ошибочно принять нулевую гипотезу) 43.7%, что гораздо больше допустимых 20%
- об этом же говорит нам мощность эксперимента недостаточная мощность эксперимента 56.3%, она должна быть больше 80% что бы вероятност совершить ошибку второго рода была на допустимо низком уровне.
- количество наблюдений. Для того, что бы результаты эксперимента были доастаточно надежными, нужно минимум 19 148 наблюдений. У нас по результатам эксперимента количество наблюдений сильно ниже.
Таким образом, по результатам А/В теста мы не должны раскатывать изменения на всех пользователей. Т.к. количество наблюдений недостаточно, вероятно нужно проверить качество сплитования с помощью А/А теста и повторить эксперимент набрав нужное количество набдюдений (если это возможно).